In [1]:
import numpy as np
import os
import sys

In [2]:
def write_inlist(base_inlist, new_inlist_file, parameter_names, parameter_values, inlist_number, storage_directory='/mnt/sdb3/neutron_star_code_data/basic_run/training_grid/'):
    
    new_inlist = open(new_inlist_file, 'w')
    
    base_inlist.seek(0)
    
    for line in base_inlist.readlines():
    
        parameter_line = False
        for i, parameter_name in enumerate(parameter_names):
            if line[:7+len(parameter_name)] == '    {} = '.format(parameter_name):
                new_inlist.write('    {} = {}\n'.format(parameter_name, parameter_values[i]))
                parameter_line = True
                break
                
        if parameter_line == True:
            continue
        
        elif line[:23] == '    output_directory = ':
            new_inlist.write("\n    output_directory = '{}LOGS{:04d}'\n".format(storage_directory, inlist_number))

        # If it is not one of the lines to change, then copy it to each new inlist.
        else:
            new_inlist.write(line)
            
    new_inlist.close()

In [3]:
def write_inlist2(base_inlist, new_inlist_file, parameter_names, parameter_values, inlist_number, storage_directory='/mnt/sdb3/neutron_star_code_data/basic_run/training_grid/'):
    
    # Create a new inlist at the location of the filename given.
    new_inlist = open(new_inlist_file, 'w')
    
    # Move to the beginning of the base inlist to copy line-by-line.
    base_inlist.seek(0)
    
    # Iterate through the base inlist line-by-line.
    for line in base_inlist.readlines():
        # If the first character of the line is a '/', we have reached the end of the document and will break from the loop without writing the line.
        if line[0] =='/':
            break
        # If the line contains an assignment for one of the variable parameters or the output directory location, we will skip it.
        # We will write these assignments at the end.
        elif any('    {} = '.format(parameter_name) in line for parameter_name in parameter_names) or '    output_directory = ' in line:
            continue
        else:
            new_inlist.write(line)
        
    # Write the output directory location assignment line.
    new_inlist.write("\n    output_directory = '{}LOGS{:04d}'\n".format(storage_directory, inlist_number))
    
    # Write each variable parameter value assignment line.
    for i, parameter_name in enumerate(parameter_names):
        new_inlist.write('    {} = {}\n'.format(parameter_name, parameter_values[i]))
        
    new_inlist.write('/')
            
    new_inlist.close()

In [4]:
def create_inlist_set(set_parameters, parameter_names, storage_directory, base_inlist_file='base_inlist'):
    # Open a base inlist that includes all parameters that will stay constant across all runs.
    # We will add the variable parameters to this inlist.
    base_inlist = open(base_inlist_file, 'r')
    
    # Iterate through each set of parameter values for all models.
    for i, parameter_values in enumerate(set_parameters):
    
        # Create a numbered LOGS directory in which to save the inlist and where the LOGS files will go.
        LOGS_directory = '{}LOGS{:04d}/'.format(storage_directory, i)
        try:
            os.mkdir(LOGS_directory)
        except FileExistsError:
            pass

        # Create a new file to write the inlist based on the base_inlist.
        new_inlist_file = '{}inlist{:04d}'.format(LOGS_directory, i)
        write_inlist(base_inlist, new_inlist_file, parameter_names, parameter_values, i)
    
    # After creating all inlists, close the base_inlist.
    base_inlist.close()

In [5]:
# Set home and dStar directories.
home_directory = os.getenv('HOME')
dStar_directory = home_directory + '/dStar/'
base_inlist_file = 'base_inlist'

In [10]:
# Location to store all the training LOGS directories
training_storage_directory = '{}/Storage/Research/neutron_star_code/basic_run/training_grid/'.format(home_directory)

# Create the directory if is does not already exist.
try:
    os.mkdir(training_storage_directory)
except FileExistsError:
    pass

In [6]:
# Create the grid of parameters for the training set.

# Name of file containing ranges of parameters for the grid
parameter_file = 'parameter_ranges'
# Load names of parameters that will be varied.
parameter_names = np.loadtxt(parameter_file, usecols=0, dtype=str)
# Columns 1-3 contain numbers in the same format as a np.linspace argument:
# core_mass 1.4 2.0 11 will produce an array np.linspace(1.4, 2.0, 11) representing the core_mass values that we include in our grid.
min_values, max_values, num_parameter_values = np.loadtxt('parameter_ranges', usecols=(1,2,3), unpack=True)

# The number of different parameter values for each parameter
num_parameter_values = num_parameter_values.astype(int)
# We will produce a model for every combination of parameter values.
# The total number of models will be the product of the number of parameter values.
total_num_models = np.prod(num_parameter_values)

# A list to store the ranges of parameter values.
# Store as a list rather than an array because different parameters may have different numbers of values so the second dimension will not necessarily be the same size for all elements.
parameter_ranges = []

for i in range(len(parameter_names)):
    parameter_ranges.append(np.linspace(min_values[i], max_values[i], num_parameter_values[i]))
    
training_parameter_grid = np.zeros((total_num_models, len(parameter_names)))

for model_number, indices in enumerate(np.ndindex(tuple(num_parameter_values))):
    
    # The array of parameters for this particular model
    parameter_values = training_parameter_grid[model_number]
    
    # Iterate through each parameter and assign its value
    for i, parameter_value in enumerate(parameter_values):
        # The indices in order correspond to each parameter that we want to assign
        # so for the ith parameter value, we find the ith parameter range array,
        # then use the ith index to determine which value in that array to assign
        parameter_values[i] = parameter_ranges[i][indices[i]]

In [12]:
# Location to store all the testing LOGS directories
testing_storage_directory = '{}/Storage/Research/neutron_star_code/basic_run/testing_grid/'.format(home_directory)

# Create the directory if is does not already exist.
try:
    os.mkdir(testing_storage_directory)
except FileExistsError:
    pass

In [13]:
testing_parameter_grid = np.zeros_like(training_parameter_grid)

for i in range(testing_parameter_grid.shape[0]):
    for j in range(testing_parameter_grid.shape[1]):
        testing_parameter_grid[i, j] = np.random.uniform(low=min_values[j], high=max_values[j])

In [14]:
print(testing_parameter_grid)

[[ 1.75821337 10.46844876]
 [ 1.95181271 19.21061689]
 [ 1.66123363 15.79985722]
 [ 1.83613308 14.85034921]
 [ 1.40655071 10.38118333]
 [ 1.57306409 12.87303335]
 [ 1.73021243 15.19640107]
 [ 1.96294049 13.96672164]
 [ 1.89716541 12.46718344]
 [ 1.76957777 19.78697993]
 [ 1.73286099 19.39095715]
 [ 1.49619795 10.50612082]
 [ 1.90757847 14.05208851]
 [ 1.5709178  16.13834796]
 [ 1.82909511 16.89988546]
 [ 1.71446633 17.57530067]
 [ 1.93089851 11.56409627]
 [ 1.85371345 15.13939108]
 [ 1.92237777 15.88809051]
 [ 1.42927988 13.58087686]
 [ 1.42898177 16.57194562]
 [ 1.64456033 11.66979651]
 [ 1.65988597 18.62074346]
 [ 1.751388   15.9901542 ]
 [ 1.84077289 11.80239049]
 [ 1.59915063 15.05072964]
 [ 1.76590733 16.03655385]
 [ 1.53673717 18.41708707]
 [ 1.74821319 11.20022385]
 [ 1.58104896 18.31895141]
 [ 1.4406468  13.05832811]
 [ 1.82664806 18.00677228]
 [ 1.92046987 13.94623183]
 [ 1.98403255 19.12262816]
 [ 1.9733528  10.38233281]
 [ 1.81471912 16.85827366]
 [ 1.53234489 14.70360595]
 

In [18]:
create_inlist_set(training_parameter_grid, parameter_names, training_storage_directory)
create_inlist_set(testing_parameter_grid, parameter_names, testing_storage_directory)